<a href="https://colab.research.google.com/github/Komorebiiiiiiii/Colab-d2l/blob/main/chapter_preliminaries/pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [1]:
!pip install -U mxnet-cu101==1.7.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.0/846.0 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 86.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.21
    Uninstalling graphviz-0.21:
      Successfully uninstalled graphviz-0.21
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


# 数据预处理
:label:`sec_pandas`

为了能用深度学习来解决现实世界的问题，我们经常从预处理原始数据开始，
而不是从那些准备好的张量格式数据开始。
在Python中常用的数据分析工具中，我们通常使用`pandas`软件包。
像庞大的Python生态系统中的许多其他扩展包一样，`pandas`可以与张量兼容。
本节我们将简要介绍使用`pandas`预处理原始数据，并将原始数据转换为张量格式的步骤。
后面的章节将介绍更多的数据预处理技术。

## 读取数据集

举一个例子，我们首先(**创建一个人工数据集，并存储在CSV（逗号分隔值）文件**)
`../data/house_tiny.csv`中。
以其他格式存储的数据也可以通过类似的方式进行处理。
下面我们将数据集按行写入CSV文件中。


In [1]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

要[**从创建的CSV文件中加载原始数据集**]，我们导入`pandas`包并调用`read_csv`函数。该数据集有四行三列。其中每行描述了房间数量（“NumRooms”）、巷子类型（“Alley”）和房屋价格（“Price”）。


In [2]:
# 如果没有安装pandas，只需取消对以下行的注释来安装pandas
# !pip install pandas
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


## 处理缺失值

注意，“NaN”项代表缺失值。
[**为了处理缺失的数据，典型的方法包括*插值法*和*删除法*，**]
其中插值法用一个替代值弥补缺失值，而删除法则直接忽略缺失值。
在(**这里，我们将考虑插值法**)。

通过位置索引`iloc`，我们将`data`分成`inputs`和`outputs`，
其中前者为`data`的前两列，而后者为`data`的最后一列。
对于`inputs`中缺少的数值，我们用同一列的均值替换“NaN”项。numeric_only=1表示只对数值进行均值计算


In [7]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(inputs.mean(numeric_only=True))
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


[**对于`inputs`中的类别值或离散值，我们将“NaN”视为一个类别。**]
由于“巷子类型”（“Alley”）列只接受两种类型的类别值“Pave”和“NaN”，
`pandas`可以自动将此列转换为两列“Alley_Pave”和“Alley_nan”。
巷子类型为“Pave”的行会将“Alley_Pave”的值设置为1，“Alley_nan”的值设置为0。
缺少巷子类型的行会将“Alley_Pave”和“Alley_nan”分别设置为0和1。


In [8]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0        True      False
1       2.0       False       True
2       4.0       False       True
3       3.0       False       True


## 转换为张量格式

[**现在`inputs`和`outputs`中的所有条目都是数值类型，它们可以转换为张量格式。**]
当数据采用张量格式后，可以通过在 :numref:`sec_ndarray`中引入的那些张量函数来进一步操作。


In [11]:
#from mxnet import np
import numpy as np
X, y = np.array(inputs.to_numpy(dtype=float)), np.array(outputs.to_numpy(dtype=float))
X, y

(array([[3., 1., 0.],
        [2., 0., 1.],
        [4., 0., 1.],
        [3., 0., 1.]]),
 array([127500., 106000., 178100., 140000.]))

## 小结

* `pandas`软件包是Python中常用的数据分析工具中，`pandas`可以与张量兼容。
* 用`pandas`处理缺失的数据时，我们可根据情况选择用插值法和删除法。

## 练习

创建包含更多行和列的原始数据集。

1. 删除缺失值最多的列。
2. 将预处理后的数据集转换为张量格式。


In [13]:
#import numpy
#import pandas as pd
#import os
import os

# 创建目录和文件
os.makedirs(os.path.join('..', 'data'), exist_ok=True)
dataFile = os.path.join('..', 'data', 'house.csv')

with open(dataFile, 'w') as f:
    # 写入列名（新增SquareFeet, YearBuilt, Garage三列）
    f.write('NumRooms,Alley,SquareFeet,YearBuilt,Garage,Price\n')

    # 写入10行数据（包含各种缺失值组合）
    f.write('3,Pave,1200,1990,1,127500\n')
    f.write('NA,NA,1500,2005,2,250000\n')
    f.write('4,NA,1800,NA,1,NA\n')
    f.write('2,Pave,NA,1985,0,98000\n')
    f.write('5,NA,2200,2010,2,320000\n')
    f.write('NA,Gravel,1600,1995,1,142000\n')
    f.write('3,NA,NA,2000,NA,165000\n')
    f.write('4,Pave,2000,NA,2,275000\n')
    f.write('NA,NA,1700,2015,1,188000\n')
    f.write('2,Gravel,1100,1975,0,75000\n')

1.删除缺失值最多的列

In [14]:
#读入数据文件
data1 = pd.read_csv(dataFile)
print(data1)

   NumRooms   Alley  SquareFeet  YearBuilt  Garage     Price
0       3.0    Pave      1200.0     1990.0     1.0  127500.0
1       NaN     NaN      1500.0     2005.0     2.0  250000.0
2       4.0     NaN      1800.0        NaN     1.0       NaN
3       2.0    Pave         NaN     1985.0     0.0   98000.0
4       5.0     NaN      2200.0     2010.0     2.0  320000.0
5       NaN  Gravel      1600.0     1995.0     1.0  142000.0
6       3.0     NaN         NaN     2000.0     NaN  165000.0
7       4.0    Pave      2000.0        NaN     2.0  275000.0
8       NaN     NaN      1700.0     2015.0     1.0  188000.0
9       2.0  Gravel      1100.0     1975.0     0.0   75000.0


In [15]:
#找到每列的nan数
nan_count = data1.isna().sum()
print(nan_count)

NumRooms      3
Alley         5
SquareFeet    2
YearBuilt     2
Garage        1
Price         1
dtype: int64


In [16]:
#找到nan_count中最大数的索引
max_index = nan_count.idxmax()
print(max_index)

Alley


In [17]:
#删除nana最大的列
data1 = data1.drop(max_index, axis=1)#此处axis=1表示删除列，也可替换为axis='columns'
print(data1)

   NumRooms  SquareFeet  YearBuilt  Garage     Price
0       3.0      1200.0     1990.0     1.0  127500.0
1       NaN      1500.0     2005.0     2.0  250000.0
2       4.0      1800.0        NaN     1.0       NaN
3       2.0         NaN     1985.0     0.0   98000.0
4       5.0      2200.0     2010.0     2.0  320000.0
5       NaN      1600.0     1995.0     1.0  142000.0
6       3.0         NaN     2000.0     NaN  165000.0
7       4.0      2000.0        NaN     2.0  275000.0
8       NaN      1700.0     2015.0     1.0  188000.0
9       2.0      1100.0     1975.0     0.0   75000.0


2.将预处理的数据集转换为张量格式

In [21]:
#处理输入输出
inputs,outputs=data1.iloc[:,0:4],data1.iloc[:,4]
inputs,outputs

(   NumRooms  SquareFeet  YearBuilt  Garage
 0       3.0      1200.0     1990.0     1.0
 1       NaN      1500.0     2005.0     2.0
 2       4.0      1800.0        NaN     1.0
 3       2.0         NaN     1985.0     0.0
 4       5.0      2200.0     2010.0     2.0
 5       NaN      1600.0     1995.0     1.0
 6       3.0         NaN     2000.0     NaN
 7       4.0      2000.0        NaN     2.0
 8       NaN      1700.0     2015.0     1.0
 9       2.0      1100.0     1975.0     0.0,
 0    127500.0
 1    250000.0
 2         NaN
 3     98000.0
 4    320000.0
 5    142000.0
 6    165000.0
 7    275000.0
 8    188000.0
 9     75000.0
 Name: Price, dtype: float64)

In [23]:
#使用插值法填补输入空值
inputs=inputs.fillna(inputs.mean(numeric_only=True))
outputs=outputs.fillna(outputs.mean(numeric_only=True))
inputs,outputs

(   NumRooms  SquareFeet  YearBuilt    Garage
 0  3.000000      1200.0   1990.000  1.000000
 1  3.285714      1500.0   2005.000  2.000000
 2  4.000000      1800.0   1996.875  1.000000
 3  2.000000      1637.5   1985.000  0.000000
 4  5.000000      2200.0   2010.000  2.000000
 5  3.285714      1600.0   1995.000  1.000000
 6  3.000000      1637.5   2000.000  1.111111
 7  4.000000      2000.0   1996.875  2.000000
 8  3.285714      1700.0   2015.000  1.000000
 9  2.000000      1100.0   1975.000  0.000000,
 0    127500.000000
 1    250000.000000
 2    182277.777778
 3     98000.000000
 4    320000.000000
 5    142000.000000
 6    165000.000000
 7    275000.000000
 8    188000.000000
 9     75000.000000
 Name: Price, dtype: float64)

In [ ]:
'''
#该数据集没有离散值或类型值，无需pandas的dummy方法
inputs = pd.get_dummies(inputs,dummy_na=True)
print(inputs)
'''

In [26]:
#将输入输出转化为张量格式
import torch
X,y=torch.tensor(inputs.values),torch.tensor(outputs.values)
# X,y=np.array(inputs.to_numpy(dtype=float)),np.array(outputs.to_numpy(dtype=float))
X,y

(tensor([[3.0000e+00, 1.2000e+03, 1.9900e+03, 1.0000e+00],
         [3.2857e+00, 1.5000e+03, 2.0050e+03, 2.0000e+00],
         [4.0000e+00, 1.8000e+03, 1.9969e+03, 1.0000e+00],
         [2.0000e+00, 1.6375e+03, 1.9850e+03, 0.0000e+00],
         [5.0000e+00, 2.2000e+03, 2.0100e+03, 2.0000e+00],
         [3.2857e+00, 1.6000e+03, 1.9950e+03, 1.0000e+00],
         [3.0000e+00, 1.6375e+03, 2.0000e+03, 1.1111e+00],
         [4.0000e+00, 2.0000e+03, 1.9969e+03, 2.0000e+00],
         [3.2857e+00, 1.7000e+03, 2.0150e+03, 1.0000e+00],
         [2.0000e+00, 1.1000e+03, 1.9750e+03, 0.0000e+00]], dtype=torch.float64),
 tensor([127500.0000, 250000.0000, 182277.7778,  98000.0000, 320000.0000,
         142000.0000, 165000.0000, 275000.0000, 188000.0000,  75000.0000],
        dtype=torch.float64))

[Discussions](https://discuss.d2l.ai/t/1749)
